In [31]:
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas
import numpy as np

import matplotlib.pyplot as plt
import analysis
reload(analysis)
import calibration
reload(calibration)

NUCLEAR_HALFLIFE = 98.3
NUCLEAR_HALFLIFE_ERROR = .3

BINS = 100

data = pandas.read_csv('data/lifetime_bigger.csv')

def run_analysis(MIN_RANGE, MAX_RANGE):
    
    fitted_data, popt, pcov = analysis.fit_exponent(data, MIN_RANGE, MAX_RANGE)

    fig = plt.figure()
    ax = fig.add_subplot(211, axisbg='w')

    fitted_data.plot(
        x='Channel',
        y='Counts',
        ax=ax
    )

    fitted_data.plot(
        x='Channel',
        y='Fit',
        ax=ax
    )
    fig.set_size_inches(10, 12)
    
    ax = fig.add_subplot(223, axisbg='w')
    fitted_data.plot(
        x='Channel',
        y='Residuals',
        ax=ax
    )
    
    ax = fig.add_subplot(224, axisbg='w')
    
    fitted_data['Residuals'].hist(
        ax=ax,
        bins=BINS
    )
    
    y, divisions = np.histogram(fitted_data['Residuals'], bins=BINS)
    x = []
    for i in range(len(divisions) - 1):
        x.append((divisions[i] + divisions[i+1])/2)
    
    gpopt, gpcov = analysis.fit_gaussian(x, y)
    
    gperrs = np.sqrt(np.diag(gpcov))
    
    print 'Guassian fit for residuals: gpopt, gperrs

    plt.plot()
    
    rsquared = analysis.calc_rsquared(fitted_data['Residuals'], fitted_data['Channel'], fitted_data['Counts'])
    print 'R-squared: {}'.format(rsquared)
    
    perrs = np.sqrt(np.diag(pcov))
#     print 'popt: {}\nerrors: {}'.format(popt, perrs)
    
    (m, b), (merr, berr) = calibration.calibrate()
#     print 'slope: {} +/- {}'.format(m, merr)
#     print 'intercept: {} +/- {}'.format(b, berr)
    
    total_error = 0;
    channel_percent_error = perrs[1]/popt[1]
    
    fitted_data['Time'] = fitted_data['Channel'].apply(lambda x:m*x + b)
    channel_lifetime = 1/popt[1]
    channel_error = channel_lifetime * channel_percent_error
#     print 'Lifetime: {} +/- {} channels'.format(channel_lifetime, channel_error)
    lifetime = m*channel_lifetime + b
    lifetime_error = analysis.calc_lifetime_error(m, merr, channel_lifetime, channel_error, b, berr)
    print 'Lifetime: {} +/- {} ns'.format(lifetime*1000, lifetime_error*1000)
    print 'Literature Lifetime {} +/- {} ns'.format(NUCLEAR_HALFLIFE/np.log(2), NUCLEAR_HALFLIFE_ERROR/np.log(2))
    

    
interact_manual(run_analysis, MIN_RANGE=widgets.IntSlider(min=0,max=8000,step=25,value=1200), MAX_RANGE=widgets.IntSlider(min=1000,max=8000,step=25,value=5000));

A Jupyter Widget